In [1]:
import glob
import os
import sys
try:
    sys.path.append(glob.glob('/opt/carla-simulator/PythonAPI/carla/dist/carla-*3.7-%s.egg' % (
        'win-amd64' if os.name == 'nt' else 'linux-x86_64'))[0])
except IndexError:
    pass
import carla
import random
import time

In [2]:
client = carla.Client('localhost', 2000)
client.set_timeout(2.0)

In [4]:
world = client.get_world()

In [5]:
world

In [6]:
blueprint_library = world.get_blueprint_library()

In [7]:
bp = random.choice(blueprint_library.filter('vehicle'))

In [8]:
if bp.has_attribute('color'):
    color = random.choice(bp.get_attribute('color').recommended_values)
    bp.set_attribute('color', color)

In [9]:
transform = random.choice(world.get_map().get_spawn_points())

In [10]:
vehicle = world.spawn_actor(bp, transform)

In [11]:
 vehicle.set_autopilot(True)

In [12]:
camera_bp = blueprint_library.find('sensor.camera.depth')
camera_transform = carla.Transform(carla.Location(x=1.5, z=2.4))
camera = world.spawn_actor(camera_bp, camera_transform, attach_to=vehicle)

In [14]:
cc = carla.ColorConverter.LogarithmicDepth
camera.listen(lambda image: image.save_to_disk('_out/%06d.png' % image.frame, cc))

In [13]:
print('created %s' % camera.type_id)

created sensor.camera.depth


In [16]:
location = vehicle.get_location()
location.x += 40
vehicle.set_location(location)
print('moved vehicle to %s' % location)

moved vehicle to Location(x=122.249550, y=-200.389679, z=-0.005216)


In [21]:
transform.location += carla.Location(x=40, y=-3.2)
transform.rotation.yaw = -180.0
for _ in range(0, 10):
    transform.location.x += 8.0

    bp = random.choice(blueprint_library.filter('vehicle'))

    # This time we are using try_spawn_actor. If the spot is already
    # occupied by another object, the function will return None.
    npc = world.try_spawn_actor(bp, transform)
    if npc is not None:
        npc.set_autopilot()
        print('created %s' % npc.type_id)

created vehicle.bmw.isetta
created vehicle.lincoln.mkz2017
created vehicle.diamondback.century
created vehicle.toyota.prius
created vehicle.audi.tt
created vehicle.yamaha.yzf
created vehicle.toyota.prius
created vehicle.bmw.grandtourer
created vehicle.volkswagen.t2
